# <cente>PML_Lab 10: Patients Physical Activities prediction using Boosting</center>

### Step 1: Understand Data

In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import precision_score, recall_score,accuracy_score,roc_auc_score,classification_report
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [5]:
df = pd.read_csv("Human_Activity_Data.csv")

In [6]:
df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,STANDING


In [16]:
df.shape


(10299, 562)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10299 entries, 0 to 10298
Columns: 562 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), object(1)
memory usage: 44.2+ MB


In [21]:
df.columns

Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'fBodyBodyGyroJerkMag-skewness()', 'fBodyBodyGyroJerkMag-kurtosis()',
       'angle(tBodyAccMean,gravity)', 'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'Activity'],
      dtype='object', length=562)

In [24]:
type(df)

pandas.core.frame.DataFrame

In [25]:
df['Activity'].value_counts

<bound method IndexOpsMixin.value_counts of 0                  STANDING
1                  STANDING
2                  STANDING
3                  STANDING
4                  STANDING
5                  STANDING
6                  STANDING
7                  STANDING
8                  STANDING
9                  STANDING
10                 STANDING
11                 STANDING
12                 STANDING
13                 STANDING
14                 STANDING
15                 STANDING
16                 STANDING
17                 STANDING
18                 STANDING
19                 STANDING
20                 STANDING
21                 STANDING
22                 STANDING
23                 STANDING
24                 STANDING
25                 STANDING
26                 STANDING
27                  SITTING
28                  SITTING
29                  SITTING
                ...        
10269      WALKING_UPSTAIRS
10270      WALKING_UPSTAIRS
10271      WALKING_UPSTAIRS
1027

### **Step 2: Build a small dataset**

In [29]:
lay = df.loc[df['Activity'] == "LAYING"][:500]
sit = df.loc[df['Activity'] == "SITTING"][:500]
walk = df.loc[df['Activity'] == "WALKING"][:500]

frames = [lay, sit, walk]

df_new = pd.concat(frames)

In [30]:
df_new.shape

(1500, 562)

In [31]:
df_new.to_csv("Human_Activity_sample.csv")

In [32]:
df1= pd.read_csv('Human_Activity_sample.csv')

In [33]:
df1.head()

,Unnamed: 0,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,51,0.403474,-0.015074,-0.118167,-0.914811,-0.895231,-0.891748,-0.917696,-0.924624,-0.905895,...,0.383907,-0.021764,-0.068580,-0.043105,-0.332249,0.487505,0.432479,-0.553351,-0.432876,LAYING
1,52,0.278373,-0.020561,-0.096825,-0.984883,-0.991118,-0.982112,-0.987985,-0.990362,-0.982189,...,-0.687636,-0.912512,-0.113048,0.223038,-0.393075,0.359037,0.399853,-0.550364,-0.427593,LAYING
2,53,0.276555,-0.017869,-0.107621,-0.994195,-0.996372,-0.995615,-0.994901,-0.996360,-0.996628,...,-0.519084,-0.813040,0.293145,-0.018635,-0.246980,0.453923,0.399066,-0.546725,-0.431244,LAYING
3,54,0.279575,-0.017276,-0.109481,-0.996135,-0.995812,-0.998689,-0.996393,-0.995474,-0.999006,...,-0.855139,-0.984028,0.240526,0.149804,0.393611,0.386719,0.399555,-0.546477,-0.431646,LAYING
4,55,0.276527,-0.016819,-0.107983,-0.996775,-0.997256,-0.995422,-0.997167,-0.997108,-0.995739,...,-0.852171,-0.954254,-0.347519,-0.469155,0.308855,-0.354146,0.398889,-0.546712,-0.431207,LAYING


In [35]:
df1.shape

(1500, 563)

In [37]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Columns: 563 entries, Unnamed: 0 to Activity
dtypes: float64(561), int64(1), object(1)
memory usage: 6.4+ MB


In [38]:
df1.columns

Index(['Unnamed: 0', 'tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y',
       'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y',
       'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y',
       'tBodyAcc-mad()-Z',
       ...
       'fBodyBodyGyroJerkMag-skewness()', 'fBodyBodyGyroJerkMag-kurtosis()',
       'angle(tBodyAccMean,gravity)', 'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'Activity'],
      dtype='object', length=563)

In [39]:
type(df1)

pandas.core.frame.DataFrame

In [41]:
df1["Activity"].value_counts()

SITTING    500
LAYING     500
WALKING    500
Name: Activity, dtype: int64

### Step 3: Build GradientBoostingClassifier

In [42]:
X=df1.drop('Activity',axis=1)
y=df1.Activity

In [43]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [44]:
model = GradientBoostingClassifier(n_estimators=100,learning_rate=1.0,max_depth=1,random_state=0)
model.fit(X_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=1.0, loss='deviance', max_depth=1,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

In [46]:
y_pred=model.predict(X_test)

In [47]:
accuracy_score(y_test,y_pred)

1.0

In [51]:
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

     LAYING       1.00      1.00      1.00       148
    SITTING       1.00      1.00      1.00       141
    WALKING       1.00      1.00      1.00       161

avg / total       1.00      1.00      1.00       450



### Step4. [Find Best no. of trees and Best Learning Rate using Grid Search and Cross Validation]

In [52]:
classifier = GradientBoostingClassifier()

In [53]:
all_scores = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=5)

In [54]:
all_scores

array([1.        , 1.        , 1.        , 1.        , 0.99519231])

### To find the average of all the accuracies, simple use the mean() method

In [55]:
all_scores.mean()

0.9990384615384615

In [56]:
parameter = {'n_estimators': [50, 100, 200, 400], 'learning_rate': [0.1, 0.01]}

In [57]:
model1 = GridSearchCV(estimator=classifier, param_grid=parameter,cv=5, n_jobs=-1)

In [58]:
model1.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 200, 400], 'learning_rate': [0.1, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [81]:
y_pred2=model1.predict(X_test)

In [82]:
accuracy_score(y_test,y_pred2)

1.0

In [83]:
print(classification_report(y_test,y_pred2))

             precision    recall  f1-score   support

     LAYING       1.00      1.00      1.00       148
    SITTING       1.00      1.00      1.00       141
    WALKING       1.00      1.00      1.00       161

avg / total       1.00      1.00      1.00       450



In [62]:
print(model1.best_estimator_)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=50,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)


### Step5. [Build AdaBoostClassifier]

In [63]:
base = DecisionTreeClassifier()

In [64]:
model2 = AdaBoostClassifier(base_estimator=base,random_state=0)

In [65]:
param_grid = {'n_estimators': [100, 150, 200], 'learning_rate': [0.01, 0.001]}

In [66]:
model3 = GridSearchCV(model2,param_grid,cv=5,n_jobs=-1)

In [67]:
model3.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=50, random_state=0),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [100, 150, 200], 'learning_rate': [0.01, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [68]:
y_pred3=model3.predict(X_test)

In [69]:
accuracy_score(y_test,y_pred3)

0.9977777777777778

In [70]:
print(classification_report(y_test,y_pred3))

             precision    recall  f1-score   support

     LAYING       1.00      1.00      1.00       148
    SITTING       1.00      0.99      1.00       141
    WALKING       0.99      1.00      1.00       161

avg / total       1.00      1.00      1.00       450



In [71]:
print(model3.best_estimator_)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.01, n_estimators=100, random_state=0)


### Step6. [Build LogisticRegressionCV classifier]

In [72]:
model4 = LogisticRegressionCV(cv=4,Cs=5,penalty='l2')

In [73]:
model4.fit(X_train,y_train)

LogisticRegressionCV(Cs=5, class_weight=None, cv=4, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [77]:
y_pred4=model4.predict(X_test)

In [84]:
accuracy_score(y_test,y_pred4)

1.0

In [85]:
print(classification_report(y_test,y_pred4))

             precision    recall  f1-score   support

     LAYING       1.00      1.00      1.00       148
    SITTING       1.00      1.00      1.00       141
    WALKING       1.00      1.00      1.00       161

avg / total       1.00      1.00      1.00       450



### Step 7 [ Build VotingClassifier]

In [89]:
model5=VotingClassifier(estimators=[('lr',model4),('gbc',model1)], voting='hard')

In [90]:
model5.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', VotingClassifier(estimators=[('lr', LogisticRegressionCV(Cs=5, class_weight=None, cv=4, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, sol...   pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [91]:
y_pred5=model5.predict(X_test)

In [92]:
print(classification_report(y_test,y_pred5))

             precision    recall  f1-score   support

     LAYING       1.00      1.00      1.00       148
    SITTING       1.00      1.00      1.00       141
    WALKING       1.00      1.00      1.00       161

avg / total       1.00      1.00      1.00       450



### Step8. [ Interpret your results]

In [93]:
print(model1.best_estimator_)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=50,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)


In [94]:
print(model3.best_estimator_)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.01, n_estimators=100, random_state=0)


## GradientBoostingClassifier

### GradientBoostingClassifier(n_estimators=50)

In [95]:
classifierF = GradientBoostingClassifier(n_estimators=50)
all_scoresF = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=5)
parameter = {'n_estimators': [50, 100, 200, 400], 'learning_rate': [0.1, 0.01]}

In [96]:
modelGC = GridSearchCV(estimator=classifier, param_grid=parameter,cv=5, n_jobs=-1)

In [97]:
modelGC.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 200, 400], 'learning_rate': [0.1, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [98]:
y_predGC=model3.predict(X_test)

In [99]:
accuracy_score(y_test,y_predGC)

0.9977777777777778

In [100]:
print(classification_report(y_test,y_predGC))

             precision    recall  f1-score   support

     LAYING       1.00      1.00      1.00       148
    SITTING       1.00      0.99      1.00       141
    WALKING       0.99      1.00      1.00       161

avg / total       1.00      1.00      1.00       450



## AdaBoostClassifier

### AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), learning_rate=0.01,n_estimators=100, random_state=0)

In [101]:
modelABC = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), learning_rate=0.01,n_estimators=100, random_state=0)

In [102]:
param_grid = {'n_estimators': [100, 150, 200], 'learning_rate': [0.01, 0.001]}

In [103]:
modelGSCV = GridSearchCV(modelABC,param_grid,cv=5,n_jobs=-1)
modelGSCV.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.01, n_estimators=100, random_state=0),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [100, 150, 200], 'learning_rate': [0.01, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [104]:
y_predGSCV=model3.predict(X_test)

In [105]:
accuracy_score(y_test,y_predGSCV)

0.9977777777777778

In [106]:
print(classification_report(y_test,y_predGSCV))

             precision    recall  f1-score   support

     LAYING       1.00      1.00      1.00       148
    SITTING       1.00      0.99      1.00       141
    WALKING       0.99      1.00      1.00       161

avg / total       1.00      1.00      1.00       450

